In [2]:
import os
import gc
import re
import sys
import time
import json
import math
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv('../tmp/train.csv')
train.head()

,id,car_application,city,created_on,domain,mile,month,price,title,transfer_owner,year,brand_slug,model_slug,detail_slug,online_year,price_bn,brand_name,model_name,detail_name,province
0,858413,非营运,北京,2018-06-09 00:00:37,guazi.com,15.49,5,7.50,ds42012款1.6t风尚版(进口),0,2014,169.0,2078.0,13136.0,2012.0,24.28,DS,DS 4,2012款 1.6T 风尚版,北京
1,872391,非营运,南充,2018-09-22 17:01:37,guazi.com,7.31,6,6.60,ds42012款1.6t风尚版(进口),0,2014,169.0,2078.0,13136.0,2012.0,24.28,DS,DS 4,2012款 1.6T 风尚版,四川
2,343319,非营运,宜昌,2018-07-29 15:40:13,guazi.com,3.51,4,8.44,ds42012款1.6t风尚版(进口),0,2014,169.0,2078.0,13136.0,2012.0,24.28,DS,DS 4,2012款 1.6T 风尚版,湖北
3,207696,非营运,深圳,2018-07-25 10:41:33,guazi.com,2.34,5,10.00,ds42012款1.6t风尚版(进口),0,2014,169.0,2078.0,13136.0,2012.0,24.28,DS,DS 4,2012款 1.6T 风尚版,广东
4,471550,非营运,成都,2018-09-28 19:58:46,guazi.com,2.43,9,10.00,ds42012款1.6t风尚版(进口),0,2014,169.0,2078.0,13136.0,2012.0,24.28,DS,DS 4,2012款 1.6T 风尚版,四川


In [8]:
data = train.loc[(train['created_on'] >= '2018-09-01'), :]
data = data.sort_values(by=['online_year','price_bn','year']).reset_index(drop=True)
data['price'] = data['price']*10000
data['price'] = data['price'].astype(int)
data = data.loc[:, ['id','created_on','province','city','month','mile','brand_slug','model_slug','detail_slug','brand_name','model_name',
                   'detail_name','price_bn','online_year', 'year','price']]
data.to_csv('../tmp/wait_predict.csv', index=False)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251086 entries, 0 to 251085
Data columns (total 16 columns):
id             251086 non-null int64
created_on     251086 non-null object
province       251086 non-null object
city           251086 non-null object
month          251086 non-null int64
mile           251086 non-null float64
brand_slug     251086 non-null float64
model_slug     251086 non-null float64
detail_slug    251086 non-null float64
brand_name     251086 non-null object
model_name     251086 non-null object
detail_name    251086 non-null object
price_bn       251086 non-null float64
online_year    251086 non-null float64
year           251086 non-null int64
price          251086 non-null int64
dtypes: float64(6), int64(4), object(6)
memory usage: 30.7+ MB


In [7]:
predict_result = pd.read_csv('../tmp/predict_result.csv')
predict_result.head()

,id,province,city,month,mile,brand_slug,model_slug,detail_slug,brand_name,model_name,detail_name,price_bn,online_year,year,price,buy_price
0,46197,山东,青岛,6,10.51,1.0,16.0,1846.0,大众,捷达,2005款 CIF 伙伴,7.68,2005.0,2006,9600,11089.0
1,936172,广东,汕头,11,24.30,1.0,16.0,1846.0,大众,捷达,2005款 CIF 伙伴,7.68,2005.0,2006,8000,10777.0
2,261194,辽宁,抚顺,11,17.15,1.0,16.0,1846.0,大众,捷达,2005款 CIF 伙伴,7.68,2005.0,2006,11800,10850.0
3,323148,山东,烟台,6,14.96,1.0,16.0,1846.0,大众,捷达,2005款 CIF 伙伴,7.68,2005.0,2006,9600,11031.0
4,752147,山东,潍坊,3,9.60,1.0,16.0,1846.0,大众,捷达,2005款 CIF 伙伴,7.68,2005.0,2006,12000,11103.0
